## Категории, группы и теги

In [1]:
from my_package.models import Tag, TAGS_DF

TAGS_DF.head()

,group,category
tag,,
Tag.RESIDENTIAL,Group.PURPOSE,Category.SPATIAL_PARAMETERS
Tag.COMMERCIAL,Group.FUNCTIONAL_ZONE,Category.ZONING
Tag.INDUSTRIAL,Group.FUNCTIONAL_ZONE,Category.ZONING
Tag.AGRICULTURE,Group.FUNCTIONAL_ZONE,Category.ZONING
Tag.SOCIAL,Group.FUNCTIONAL_ZONE,Category.ZONING


Можем для каждой группы/категории вывести дочерние вещи

In [2]:
for category in TAGS_DF.category.unique():
  print(category.value)
  category_df = TAGS_DF[TAGS_DF.category == category]
  for group in category_df.group.unique():
    print('|- ' + group.value)
    group_df = category_df[category_df.group == group]
    for tag in group_df.index:
      print('|  |- ' + tag.value)

Пространственные параметры
|- Назначение
|  |- Жилая
|  |- Нежилая
|  |- Смешанная
|- Центральность
|  |- Центральный
|  |- Потенциально-центральный
|  |- Не центральный
|- Плотность
|  |- Плотная
|  |- Низкоплотная
|- Этажность
|  |- Высотная
|  |- Многоэтажная
|  |- Среднеэтажная
|  |- Малоэтажная
Зонирование
|- Функциональная зона
|  |- Коммерческая
|  |- Промышленная
|  |- Сельскохозяйственная
|  |- Социальная
|  |- Водно-зеленый каркас
По отношению к объекту
|- Транспорт
|  |- Общегородского значения
|  |- Районного значения
|  |- Местного значения
|- Инфраструктура
|  |- Медицинские учреждения
|  |- Образовательные учреждения
|  |- Спортивные объекты
|  |- Коммерческие объекты
|  |- Культурные объекты
|  |- Рекреационные объекты
|- Прибрежные объекты
|  |- Водный объект
|  |- Городская набережная
|  |- Пляж
|- Знаковые объекты
|  |- Доминанты и видовые объекты
|  |- Объекты культурного наследия
|  |- Городские достопримечательности


Можем для тэга получить его категорию и группу

In [3]:
tag_info = TAGS_DF.loc[Tag.AGRICULTURE]
tag_info.category, tag_info.group

(<Category.ZONING: 'Зонирование'>,
 <Group.FUNCTIONAL_ZONE: 'Функциональная зона'>)

## Матрица перехода

In [4]:
from my_package.models.transition_matrix import TRANSITION_MATRIX

TRANSITION_MATRIX

,Tag.RESIDENTIAL,Tag.COMMERCIAL,Tag.INDUSTRIAL,Tag.AGRICULTURE,Tag.SOCIAL,Tag.RECREATION
Tag.RESIDENTIAL,1.0,0.0,0.0,0.0,0.0,0.0
Tag.COMMERCIAL,0.0,0.5,0.0,0.0,0.0,0.5
Tag.INDUSTRIAL,0.1,0.3,0.5,0.0,0.1,0.0
Tag.AGRICULTURE,0.0,0.0,0.0,0.0,0.5,0.5
Tag.SOCIAL,0.0,0.0,0.0,0.0,1.0,0.0
Tag.RECREATION,0.0,0.0,0.0,0.0,0.0,1.0


## Матрица требований

In [5]:
from my_package.models.requirements_matrix import REQUIREMENTS_MATRIX

REQUIREMENTS_MATRIX.head()

,Requirement.COMMERCIAL_PLACEMENT,Requirement.INFRASTRUCTURE_PLACEMENT,Requirement.TRANSPORT_PLACEMENT,Requirement.SERVICE_PLACEMENT,Requirement.PROTECTION_ZONE,Requirement.RECREATION_ZONE,Requirement.ICONIC_OBJECT,Requirement.NEW_OBJECT,Requirement.VIEW_CHARACTERISTICS,Requirement.HEIGHT_CONTROL,Requirement.PUBLIC_SPACE
Tag.RESIDENTIAL,False,False,None,None,False,False,None,False,True,True,True
Tag.COMMERCIAL,True,True,None,None,False,False,True,True,None,None,True
Tag.INDUSTRIAL,True,True,False,None,None,None,False,None,True,None,True
Tag.AGRICULTURE,True,True,None,True,False,None,False,None,False,True,None
Tag.SOCIAL,True,False,None,False,True,True,False,False,False,True,False


## Черновик решения матрицы

Допустим, у нас выпал набор тегов

In [24]:
import random

tags = [random.choice(TAGS_DF[TAGS_DF.group == group].index) for group in TAGS_DF.group.unique()]
tags

[<Tag.RESIDENTIAL: 'Жилая'>,
 <Tag.SOCIAL: 'Социальная'>,
 <Tag.CITY_SIGNIFICANCE: 'Общегородского значения'>,
 <Tag.SPORTS_FACILITIES: 'Спортивные объекты'>,
 <Tag.CITY_EMBANKMENT: 'Городская набережная'>,
 <Tag.CULTURAL_HERITAGE: 'Объекты культурного наследия'>,
 <Tag.NON_CENTRAL: 'Не центральный'>,
 <Tag.DENSE: 'Плотная'>,
 <Tag.MULTI_STOREY: 'Многоэтажная'>]

По нему мы выбираем подматрицу

In [25]:
SUB_MATRIX = REQUIREMENTS_MATRIX.loc[tags]
SUB_MATRIX

,Requirement.COMMERCIAL_PLACEMENT,Requirement.INFRASTRUCTURE_PLACEMENT,Requirement.TRANSPORT_PLACEMENT,Requirement.SERVICE_PLACEMENT,Requirement.PROTECTION_ZONE,Requirement.RECREATION_ZONE,Requirement.ICONIC_OBJECT,Requirement.NEW_OBJECT,Requirement.VIEW_CHARACTERISTICS,Requirement.HEIGHT_CONTROL,Requirement.PUBLIC_SPACE
Tag.RESIDENTIAL,False,False,None,None,False,False,None,False,True,True,True
Tag.SOCIAL,True,False,None,False,True,True,False,False,False,True,False
Tag.CITY_SIGNIFICANCE,False,False,False,None,True,False,False,False,True,True,False
Tag.SPORTS_FACILITIES,False,False,True,False,False,True,None,True,None,False,True
Tag.CITY_EMBANKMENT,True,None,True,True,None,False,False,True,True,True,False
Tag.CULTURAL_HERITAGE,False,None,None,False,False,True,False,True,True,False,None
Tag.NON_CENTRAL,True,False,None,False,True,True,True,True,None,None,False
Tag.DENSE,True,None,False,None,None,None,None,True,True,True,None
Tag.MULTI_STOREY,True,None,False,True,True,True,False,False,None,True,True


Так примерно выглядит столбец во время разрешивания, надо подумать какая логика может сюда идти

In [26]:
column = SUB_MATRIX[SUB_MATRIX.columns[0]]
column[~column.isna()]

Tag.RESIDENTIAL          False
Tag.SOCIAL                True
Tag.CITY_SIGNIFICANCE    False
Tag.SPORTS_FACILITIES    False
Tag.CITY_EMBANKMENT       True
Tag.CULTURAL_HERITAGE    False
Tag.NON_CENTRAL           True
Tag.DENSE                 True
Tag.MULTI_STOREY          True
Name: Requirement.COMMERCIAL_PLACEMENT, dtype: object

И теперь можем попробовать для каждого столбца решить, применяем мы его или нет (правила можно и помудрить)

In [30]:
for requirement in SUB_MATRIX.columns:
  series = SUB_MATRIX[requirement] # get certain column
  series = series[~series.isna()] # filter None values for the requirement
  trues = series[series == True]
  falses = series[series == False]
  if len(trues) > len(falses):
    print(requirement.value, len(trues), len(falses))

размещение коммерческих сервисов ежедневного использования 5 4
устройство защитной зоны 4 3
создание рекреационных зон 5 3
размещение нового объекта  с учетом радиуса влияния 5 4
учет видовых характеристик 5 1
регулирование высотности 6 2
